# EOSC 410 Final Project

In [1]:
import numpy

### Thoughts on methodologies
* Can use multiple linear regression as a simple baseline model. This will be used to benchmark the performance of more complex models.
* Neural network ensemble: train multiple neural networks with different hidden layer sizes and parameters
* CNN: use convolutional neural networks to capture spatial information

### Thoughts on data preprocessing and data input
* 